In [ ]:
'''
- Project link: Spotify million song data set: https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset

In the data set song lyrics are given for nearly 57k songs. I used cosine disance to design
recommendation system that offers songs with similar lyrics. Recommendation system
takes any input as text however it works best when lyrics from database is given
as input. 
'''



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df = df.sample(5000)

In [4]:
df.drop(['link'], axis=1, inplace = True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 12655 to 52377
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   artist  5000 non-null   object
 1   song    5000 non-null   object
 2   text    5000 non-null   object
dtypes: object(3)
memory usage: 156.2+ KB


In [6]:
df.T

,12655,42538,13806,54270,15554,22150,23012,5643,5036,20540,...,40946,55615,23196,36303,42871,4871,49745,29553,23777,52377
artist,Mc Hammer,Madonna,Neil Young,Ufo,Paul McCartney,Yoko Ono,Aerosmith,Everclear,Elvis Presley,Van Morrison,...,Kyla,Waterboys,Alabama,Howard Jones,Marillion,Elton John,R. Kelly,Depeche Mode,Alphaville,Stone Roses
song,Pray,Lament,Be The Rain,Sweet Little Thing,Dance Tonight,If Only,Chiquita,White Noise,Doin' The Best I Can,Carrying A Torch,...,Old Friend,Fisherman's Blues,O Little Town Of Bethlehem,You Know I Love You... Don't You?,Exile On Princes Street,Dark Diamond,Imagine That,No Disco,Heartbreaker,Tightrope
text,"That's word,we pray(pray,pray) \r\nWe got to ...","(Eva:) \r\nThe choice was mine, and mine comp...",Save the planet for another day \r\n(Attentio...,"Just down here said hello, tell me are you try...",Everybody gonna dance tonight \r\nEverybody g...,I cut my finger when you left the room \r\nTh...,"Ooh, on the beach at cartegena \r\nShe, salty...","All the people that you know, they say you're ...",I gave my heart \r\nI gave my love \r\nOh ho...,I'm carryin' a torch for you \r\nI'm carryin'...,...,A million times or more I thought about you \...,I wish I was a fisherman \r\nTumblin' on the ...,O little town of Bethlehem how still we see th...,You got the socket and I got the plug \r\nI a...,"Due to the nature of this demo, no printed lyr...","Oh, I'm a dark diamond \r\nI've turned hard a...",Close your eyes \r\nNow relax your mind \r\n...,I saw you in the picture \r\nI saw you play t...,I feel so lonely when I'm not with you \r\nIt...,"You should've been an angel, it would've suite..."


In [7]:
# Text processing

In [8]:
df['text'].str.lower().replace(r'^\w\s','')
df['text'].str.lower().replace(r'\n','', regex = True)
# kucuk harfe donustur
df["text"]=df["text"].str.lower()
# noktalama isaretlerini kaldirma
df["text"]=df["text"].str.replace("[^\w\s]","") 
# rakamlari kaldir
df["text"]=df["text"].str.replace("\d+","") 
# \newline ve enter\r kaldir
df["text"]=df["text"].str.replace("\n"," ").replace("\r","") 
# fazladan bosluklari azalt
df["text"]=df["text"].str.replace("  "," ")

C:\Users\haluk\AppData\Local\Temp\ipykernel_13620\3634558233.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"]=df["text"].str.replace("[^\w\s]","")
C:\Users\haluk\AppData\Local\Temp\ipykernel_13620\3634558233.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text"]=df["text"].str.replace("\d+","")


In [9]:
# strip out html tags
import html
def del_html(text):
   return html.unescape(text)

# Apply to multiple columns
df['text'] = df['text'].apply(del_html)

In [10]:
import nltk
from nltk.stem.porter import PorterStemmer
from textblob import TextBlob
stemmer = PorterStemmer()

In [11]:
# correct and tokenize the text 
def token(text):
    
    #blob = TextBlob(text)
    #text = str(blob.correct())
    token = nltk.word_tokenize(text)
    tokens = [stemmer.stem(w) for w in token]
    return " ".join(tokens)

In [12]:
df['text'] = df['text'].apply(lambda x :token(x))

In [13]:
#!pip install textblob

In [14]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Define a function to remove stop words from a string
def remove_stop_words(text):

    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    filtered_text = " ".join(filtered_words)
    return filtered_text

# Apply the function to the pandas column
df['text_sw'] = df['text'].apply(remove_stop_words)

In [15]:
df['text_sw']

12655    wordw praypraypray got pray make today said pr...
42538    eva choic wa mine mine complet could ani prize...
13806    save planet anoth day attent shopper buy consc...
54270    said hello tell tri get said hey honey dont ne...
15554    everybodi gon na danc tonight everybodi gon na...
                               ...                        
4871     oh im dark diamond ive turn hard cold onc wa j...
49745    close eye relax mind want pictur far far away ...
29553    saw pictur saw play part thi aint disco thousa...
23777    feel lone im shame im spend time without yet f...
52377    shouldv angel wouldv suit goldleaf tryptic ang...
Name: text_sw, Length: 5000, dtype: object

In [16]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=10000, stop_words="english")
x =vectorizer.fit_transform(df['text_sw'])

In [18]:
x.shape

(5000, 10000)

In [19]:
similarity = cosine_similarity(x)

In [20]:
similarity[1]

array([0.00350931, 1.        , 0.07788464, ..., 0.00903412, 0.03834438,
       0.03572282])

In [21]:
df_trk = df[['text_sw']]

In [22]:
df_trk

,text_sw
12655,wordw praypraypray got pray make today said pr...
42538,eva choic wa mine mine complet could ani prize...
13806,save planet anoth day attent shopper buy consc...
54270,said hello tell tri get said hey honey dont ne...
15554,everybodi gon na danc tonight everybodi gon na...
...,...
4871,oh im dark diamond ive turn hard cold onc wa j...
49745,close eye relax mind want pictur far far away ...
29553,saw pictur saw play part thi aint disco thousa...
23777,feel lone im shame im spend time without yet f...


In [23]:
def recommend_track(input_lyrics) :
    
    # tokenize input text
    input_lyrics = token(input_lyrics)
    
    # remove stop words
    input_lyrics= remove_stop_words(input_lyrics)
    #print(input_lyrics)
    text_row = {'text_sw':input_lyrics}
    
    len_trk = len(df['text_sw'])
    df_trk = pd.DataFrame({'text_sw': range(len_trk)})
        
    df_trk = df[['text_sw']]
    
    #print(df_trk.shape)
    len_trk = len(df_trk)
    # add lyrics tokens as if new song lycrics 
    df_trk.loc[len_trk] = text_row;
    #print(df_trk.shape)
    
    x =vectorizer.fit_transform(df_trk['text_sw'])
    similarity_wInput = cosine_similarity(x)
    
    indexes = sorted(list ( enumerate(similarity_wInput[len_trk]) ),reverse = True,   key = lambda x:x[1] )[1:11]
    
    for i in indexes :
        track_indx, similarity_val = i;
        print("Cos-dist: {} {} \t{}".format(round(similarity_val, 4),df['artist'].iloc[track_indx] , df['song'].iloc[track_indx] ) )
        
    #print(new_similarity.shape)
    

In [24]:
# these are the lyrics of a song from Britney Spears 
# recommendation system finds similar songs 

recommend_track("My friends say you are so into me\
And that you need me desperately\
They say you say we are so complete\
But I need to hear it straight from you\
If you want me to believe it is true\
I have been waiting for so long it hurts\
I want to hear you say the words, please\
Don't, don't let me be the last to know\
Don't hold back, just let it go\
I need to hear you say\
You need me all the way\
Oh, if you love me so\
Don't let me be the last to know\
Ooh yeah, baby\
Your body language says so much\
Yeah, I feel it in the way you touch\
But 'til you say the words it's not enough\
Come on and tell me you're in love, please\
Don't, don't let me be the last to know\
Don't hold back, just let it go\
I need to hear you say\
You need me all the way\
Oh, if you love me so\
Don't let me be the last to know\
Come on baby, come on darling, ooh yeah\
Come on, let me be the one\
Come on now, oh yeah\
I need to hear you say (I need to hear you say)\
You love me all the way (you love me all the way)\
And I don't want to wait another day\
I want to feel the way you feel\
Oh, come on\
Don't, (don't) don't let me be the last to know (just let me be the one)\
Don't (don't) hold back (hold back), just let it go (just let it go)\
I need to hear you say (I need to hear you say)\
You need me all the way (you need me all the way)\
So, baby, if you love me\
Don't let me be the last to know")

C:\Users\haluk\AppData\Local\Temp\ipykernel_13620\4053715263.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trk.loc[len_trk] = text_row;


Cos-dist: 0.7377 Britney Spears 	Don't Let Me Be The Last To Know
Cos-dist: 0.455 Cliff Richard 	Never Let Go
Cos-dist: 0.412 Steve Miller Band 	Little Girl
Cos-dist: 0.373 Oasis 	All You Need Is Love
Cos-dist: 0.3714 Foreigner 	Don't Let Go
Cos-dist: 0.3451 Omd 	Apollo
Cos-dist: 0.3421 Ten Years After 	Let The Sky Fall
Cos-dist: 0.3388 Fatboy Slim 	Everybody Needs A 303
Cos-dist: 0.3382 Dean Martin 	Let Me Go Lover
Cos-dist: 0.3368 Norah Jones 	I Wouldn't Need You


In [36]:
recommend_track("good musich is right here")

C:\Users\haluk\AppData\Local\Temp\ipykernel_13620\4053715263.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_trk.loc[len_trk] = text_row;


Cos-dist: 0.6101 Van Halen 	You're No Good
Cos-dist: 0.4762 Styx 	Right Away
Cos-dist: 0.4056 Tina Turner 	Don't Rush The Good Things
Cos-dist: 0.3971 Clash 	Know Your Rights
Cos-dist: 0.3775 Don McLean 	Words And Music
Cos-dist: 0.3712 Chaka Khan 	Have A Good Time
Cos-dist: 0.3684 Old 97's 	St. Ignatius
Cos-dist: 0.3583 Ella Fitzgerald 	Ill Wind
Cos-dist: 0.3395 Wanda Jackson 	Right Or Wrong
Cos-dist: 0.3379 Diana Ross 	Don't Stop
